See [Capturing of the stdout/stderr output](https://docs.pytest.org/en/latest/capture.html#capturing-of-the-stdout-stderr-output)

.py files for tutorial kept at `6_capturing_stdout_stderr`

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Default-stdout/stderr/stdin-capturing-behavior" data-toc-modified-id="Default-stdout/stderr/stdin-capturing-behavior-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Default stdout/stderr/stdin capturing behavior</a></span></li><li><span><a href="#Setting-capturing-methods-or-disabling-capturing" data-toc-modified-id="Setting-capturing-methods-or-disabling-capturing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting capturing methods or disabling capturing</a></span></li><li><span><a href="#Using-print-statements-for-debugging" data-toc-modified-id="Using-print-statements-for-debugging-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Using print statements for debugging</a></span></li><li><span><a href="#Accessing-captured-output-from-a-test-function" data-toc-modified-id="Accessing-captured-output-from-a-test-function-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Accessing captured output from a test function</a></span></li></ul></div>

# Default stdout/stderr/stdin capturing behavior

See [Default stdout/stderr/stdin capturing behavior](https://docs.pytest.org/en/latest/capture.html#default-stdout-stderr-stdin-capturing-behaviour)

During test execution, any output sent to `stdout` and `stderr` is captured

Resulting captured output can be shown via `--show-capture`

Also, `stdin` is set to a "null" object which will fail on read attempts

Default capturing is done by intercepting writes to low level file descriptors

# Setting capturing methods or disabling capturing

See [Setting capturing methods or disabling capturing](https://docs.pytest.org/en/latest/capture.html#setting-capturing-methods-or-disabling-capturing)

`pytest` can perform capturing via:

* file descriptor (FD) capturing (default)
  * All writes going to the OS file descriptors 1 and 2 will be captured
* `sys` level capturing
  * Only writes to `sys.stdout` and `sys.stderr` will be captured, without any capturing of FD writes
  
Command line options:

```python
pytest -s            # disable all capturing
pytest --capture=sys # replace sys.stdout/stderr with in-mem files
pytest --capture=fd  # also point filedescriptors 1 and 2 to temp file
```

# Using print statements for debugging

Print statements can be used for debugging if `stdout`/`stderr` output is captured:

```python
# content of test_module.py

def setup_function(function):
    print ("setting up %s" % function)

def test_func1():
    assert True

def test_func2():
    assert False
```

Running the test will show the output of the failing function:

```
$ pytest -q test_module.py
.F                                                                       [100%]
================================== FAILURES ===================================
_________________________________ test_func2 __________________________________

    def test_func2():
>       assert False
E       assert False

test_module.py:10: AssertionError
---------------------------- Captured stdout setup ----------------------------
setting up <function test_func2 at 0x000001B7711316A8>
1 failed, 1 passed in 0.03 seconds
```

# Accessing captured output from a test function

See [Accessing captured output from a test function](https://docs.pytest.org/en/latest/capture.html#accessing-captured-output-from-a-test-function)

The `capsys`, `capsysbinary`, `capfd`, and `capfdbinary` fixtures allow access to stdout/stderr output from tests

An example test with output related checks:

```python
def test_myoutput(capsys):  # or use "capfd" for fd-level
    print("hello")
    sys.stderr.write("world\n")
    captured = capsys.readouterr()
    assert captured.out == "hello\n"
    assert captured.err == "world\n"
    print("next")
    captured = capsys.readouterr()
    assert captured.out == "next\n"
```

The `readouterr()` call gets the output so far and allows capturing to continue. After the function finishes, the original streams will be restored

Using `capsys` frees the test from having to care about setting/resetting output streams and also works with pytest's per-test capturing

`capfd` works on an output stream level

Text can also be disabled via the `with capsys.disabled()` context:

```
def test_disabling_capturing(capsys):
    print("this output is captured")
    with capsys.disabled():
        print("output not captured, going directly to sys.stdout")
    print("this output is also captured")
```